In [44]:
import requests
import re
import time
import math
import pandas as pd
from tqdm import tqdm
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

## UCI

In [81]:
uci_base = "https://catalogue.uci.edu"

In [82]:
uci_all = requests.get(uci_base + '/undergraduatedegrees/').text
uci_soup = BeautifulSoup(uci_all, 'html.parser')

In [112]:
uci_programs = uci_soup.select_one('h2[class="tglhead"]').find_next_siblings()[0].select('li')

In [120]:
uci_names = []
uci_reqs = []
for p in tqdm(uci_programs):
    name = p.text.split(',')
    if 'B' not in name[-1]:
        pass
    else:
        uci_names.append(''.join(name[:-1]))
        p_url = uci_base + p.a['href'] + "#requirementstext"
        p_reqs = requests.get(p_url).text
        p_soup = BeautifulSoup(p_reqs, 'html.parser')
        p_reqs_ls = list(map(lambda x: x.text.strip(), p_soup.select('td[class="codecol"] + td:not([class]):not([colspan])')))
        uci_reqs.append(', '.join(p_reqs_ls))

100%|██████████| 166/166 [00:27<00:00,  6.08it/s]


In [124]:
uci_df = pd.DataFrame({'program': uci_names, 'requirements': uci_reqs})

In [125]:
uci_df.head()

,program,requirements
0,Aerospace Engineering,"General Chemistry, Accelerated General Chemist..."
1,African American Studies,"African American Studies I, African American S..."
2,Anthropology,"Introduction to Sociocultural Anthropology, In..."
3,Applied Physics,"Single-Variable Calculus, Single-Variable Calc..."
4,Art,"Art in Context: History, Theory, and Practice,..."


## Princeton

In [170]:
pu_base = "https://ua.princeton.edu"

In [171]:
pu_ba = requests.get(pu_base + '/academic-units/departmental-concentrations-for-the-degree-of-bachelor-of-arts').text
pu_ba_soup = BeautifulSoup(pu_ba, 'html.parser')
pu_bse = requests.get(pu_base + '/academic-units/departmental-concentrations-for-the-degree-of-bachelor-of-science-in-engineering').text
pu_bse_soup = BeautifulSoup(pu_bse, 'html.parser')

In [172]:
pu_ba_programs = pu_ba_soup.select(".field-content > a")
pu_bse_programs = pu_bse_soup.select(".field-content > a")

In [227]:
pu_names = []
pu_courses = []
pu_desc = []
for p in tqdm(pu_ba_programs + pu_bse_programs):
    name = p.text.strip()
    pu_names.append(name)
    p_url = pu_base + p['href']
    p_content = requests.get(p_url).text
    p_soup = BeautifulSoup(p_content, 'html.parser')
    p_courses = ' '.join(list(map(lambda x: re.sub('\n', '', ' '.join(x.text.strip().split(' ')[2:])), p_soup.select('h3[class="course-title"]'))))
    pu_courses.append(p_courses)
    p_desc = ' '.join(list(map(lambda x: x.text.strip(), p_soup.select('.course-desc'))))
    pu_desc.append(p_desc)

100%|██████████| 37/37 [00:19<00:00,  1.95it/s]


In [228]:
pu_df = pd.DataFrame({'institution': 'Princeton', 'program': pu_names, 'courses': pu_courses, 'descriptions': pu_desc})

In [229]:
pu_df.head()

,institution,program,courses,descriptions
0,Princeton,African American Studies,African American Studies and the Philosophy of...,This course introduces students to the field o...
1,Princeton,Anthropology,Introduction to Anthropology CDEC Human Evolu...,An introduction to anthropology and key topics...
2,Princeton,Architecture,An Introduction to the History of Architecture...,A broad overview of the discipline of architec...
3,Princeton,Art and Archaeology,An Introduction to the History of Art: Meaning...,A team-taught introduction to the history of a...
4,Princeton,Astrophysical Sciences,The Universe Spring QRSN Topics in Modern Astr...,This specially designed course targets the fro...


In [230]:
pu_df.to_csv('curriculum/princeton_programs.csv', index=False)

## Texas Tech

In [7]:
# bypass SSLError
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'

In [2]:
ttu_base = "https://www.ttu.edu/majors-and-colleges/undergraduate-majors/"

In [8]:
ttu_all = requests.get(ttu_base).text
ttu_soup = BeautifulSoup(ttu_all, 'html.parser')

In [99]:
ttu_programs = ttu_soup.select('.columns-2 > ul > li > a')

In [110]:
driver = webdriver.Chrome()
ttu_names = []
ttu_courses = []
ttu_desc = []
for p in tqdm(ttu_programs):
    driver.get(p['href'])
    time.sleep(1)
    ttu_names.append(p.text)
    p_courses = driver.find_elements(By.CSS_SELECTOR, '.acalog-course > span')
    course_name_ls = []
    course_desc_ls = []
    for course in p_courses:
        a = course.find_element(By.TAG_NAME, 'a')
        course_name = re.search('(?<= - ).*', a.text).group(0)
        course_name_ls.append(course_name)
        # click on a tag and wait until element is present
        a.click()
        driver.implicitly_wait(3)
        course_desc = course.find_element(By.XPATH, 'table/tbody/tr/td/div[2]').text.split('\n')[-1]
        course_desc_ls.append(course_desc)
        time.sleep(0.5)
    ttu_courses.append(' '.join(course_name_ls))
    ttu_desc.append(' '.join(course_desc_ls))
    time.sleep(1)
driver.close()

100%|██████████| 188/188 [1:35:03<00:00, 30.34s/it]


In [111]:
ttu_df = pd.DataFrame({'institution': 'Texas Tech', 'program': ttu_names, 'courses': ttu_courses, 'descriptions': ttu_desc})

In [112]:
ttu_df.head()

,institution,program,courses,descriptions
0,Texas Tech,"Accounting, B.B.A.",Foundations of Business Introductory Mathemati...,Provides students with a basic understanding o...
1,Texas Tech,"Advertising, B.A.",Foundations of Media and Communication America...,TCCNS: [COMM1307] A broad survey of media hist...
2,Texas Tech,"Agribusiness, B.S.",Essentials of College Rhetoric Introductory Ma...,TCCNS: [ENGL1301] Prerequisite: Successful com...
3,Texas Tech,"Agricultural and Applied Economics, B.S.",Essentials of College Rhetoric Introductory Ma...,TCCNS: [ENGL1301] Prerequisite: Successful com...
4,Texas Tech,"Agricultural Communications, B.S.",Essentials of College Rhetoric College Algebra...,TCCNS: [ENGL1301] Prerequisite: Successful com...


In [114]:
ttu_df.to_csv('curriculum/texas_tech_programs.csv', index=False)

## Clemson

In [115]:
clem_base = "https://www.clemson.edu/degrees/index.html"

In [116]:
clem_all = requests.get(clem_base).text
clem_soup = BeautifulSoup(clem_all, 'html.parser')

In [121]:
clem_programs = clem_soup.select('tbody > tr')

In [159]:
clem_names = []
clem_urls = []
for p in clem_programs:
    # search for bachelor's programs only
    p_url = p.select_one('td:nth-of-type(2) > a')
    if p_url:
        p_name = p.select_one('td').text
        p_content = requests.get(p_url["href"]).text
        p_soup = BeautifulSoup(p_content, 'html.parser')
        try:
            clem_urls.append(p_soup.select_one('#curric > ul > li > a')['href'])
            clem_names.append(p_name)
        except:
            print(f'Curriculum for {p_name} not found')
            pass
    else:
        pass

Curriculum for Automotive Engineering not found
Curriculum for Modern Languages Education not found


In [ ]:
driver = webdriver.Chrome()
clem_courses = []
clem_desc = []
for p in tqdm(clem_urls):
    driver.get(p)
    time.sleep(1)
    p_courses = driver.find_elements(By.CSS_SELECTOR, '.acalog-course > span')
    course_name_ls = []
    course_desc_ls = []
    for course in p_courses:
        driver.execute_script("arguments[0].scrollIntoView(true);", course)
        a = course.find_element(By.TAG_NAME, 'a')
        course_name = re.search('(?<= - ).*', a.text).group(0)
        course_name_ls.append(course_name)
        # click on a tag and wait until element is present, bypass GUI w JS click
        driver.execute_script("arguments[0].click();", a)
        driver.implicitly_wait(3)
        course_desc = course.find_element(By.XPATH, 'table/tbody/tr/td/div[2]').text.split('\n')[-1]
        course_desc_ls.append(course_desc)
        time.sleep(0.5)
    clem_courses.append(' '.join(course_name_ls))
    clem_desc.append(' '.join(course_desc_ls))
    time.sleep(1)
driver.close()

In [198]:
clem_df = pd.DataFrame({'institution': 'Clemson', 'program': clem_names, 'courses': clem_courses, 'descriptions': clem_desc})

In [212]:
# remove Agricultural Education
clem_df = clem_df[~(clem_df['courses'] == "")]

In [213]:
clem_df.head()

,institution,program,courses,descriptions
0,Clemson,Accounting,Business Foundations Principles of Microeconom...,Introduction to a variety of topics critical t...
1,Clemson,Agribusiness,Agriculture and Society Composition and Rhetor...,Introduction to the development of world socie...
3,Clemson,Agricultural Mechanization and Business,Introduction to Agricultural Mechanization and...,Introduces the Agricultural Mechanization and ...
4,Clemson,Animal and Veterinary Sciences,Introduction to Animal Science Introduction to...,Survey of animal industries and their role in ...
5,Clemson,Anthropology,Introduction to Anthropology Essential Mathema...,Offers a four-field overview: primates and hum...


In [ ]:
clem_df.to_csv('curriculum/clemson_programs.csv', index=False)

## Purdue

In [2]:
purdue_base = "https://catalog.purdue.edu/"

In [3]:
purdue_all = requests.get(purdue_base + 'content.php?catoid=8&navoid=10451').text
purdue_soup = BeautifulSoup(purdue_all, 'html.parser')

In [4]:
purdue_undergrad = list(chain.from_iterable(map(lambda x: x.select('li > a'), purdue_soup.select('.program-list')[1:25])))
purdue_grad = list(chain.from_iterable(map(lambda x: x.select('li > a'), purdue_soup.select('.program-list')[29:53])))

In [ ]:
driver = webdriver.Chrome()
purdue_undergrad_names = []
purdue_undergrad_courses = []
purdue_undergrad_desc = []
for p in tqdm(purdue_undergrad):
    driver.get(purdue_base + p['href'])
    time.sleep(1)
    purdue_undergrad_names.append(p.text)
    p_courses = driver.find_elements(By.CSS_SELECTOR, '.acalog-course > span')
    course_name_ls = []
    course_desc_ls = []
    for course in p_courses:
        a = course.find_element(By.TAG_NAME, 'a')
        course_name = re.search('(?<= - ).*', a.text).group(0)
        course_name_ls.append(course_name)
        # click on a tag and wait until element is present
        a.click()
        driver.implicitly_wait(3)
        course_desc = course.find_element(By.XPATH, 'table/tbody/tr/td/div[2]').text.split('\n')[-1]
        course_desc_ls.append(course_desc)
        time.sleep(0.2)
    purdue_undergrad_courses.append(' '.join(course_name_ls))
    purdue_undergrad_desc.append(' '.join(course_desc_ls))
    time.sleep(0.5)
driver.close()

In [9]:
purdue_undergrad_df = pd.DataFrame({'institution': 'purdue', 'degree_level': 'undergraduate', 'program': purdue_undergrad_names, 'courses': purdue_undergrad_courses, 'descriptions': purdue_undergrad_desc})

In [14]:
purdue_undergrad_df.to_csv('curriculum/purdue_bach_programs.csv', index=False)

## University of Houston

In [15]:
uh_base = "http://publications.uh.edu/"

In [16]:
uh_all = requests.get(uh_base + 'content.php?catoid=31&navoid=11529').text
uh_soup = BeautifulSoup(uh_all, 'html.parser')

In [39]:
uh_programs = uh_soup.select('.program-list > li > a')[7:127]

In [49]:
driver = webdriver.Chrome()
uh_undergrad_names = []
uh_undergrad_courses = []
uh_undergrad_desc = []
for p in tqdm(uh_programs):
    driver.get(uh_base + p['href'])
    time.sleep(1)
    uh_undergrad_names.append(p.text)
    p_courses = driver.find_elements(By.CSS_SELECTOR, '.acalog-course > span')
    course_name_ls = []
    course_desc_ls = []
    for course in p_courses:
        a = course.find_element(By.TAG_NAME, 'a')
        course_name = re.search('(?<= - ).*', a.text).group(0)
        course_name_ls.append(course_name)
        # click on a tag and wait until element is present
        a.click()
        driver.implicitly_wait(3)
        course_desc = course.find_element(By.XPATH, 'table/tbody/tr/td/div[2]').text.split('\n')[-1]
        course_desc_ls.append(course_desc)
        time.sleep(0.2)
    uh_undergrad_courses.append(' '.join(course_name_ls))
    uh_undergrad_desc.append(' '.join(course_desc_ls))
    time.sleep(0.5)
driver.close()

100%|██████████| 120/120 [36:40<00:00, 18.34s/it]


In [50]:
uh_df = pd.DataFrame({'institution': 'University of Houston', 'degree_level': 'undergraduate', 'program': uh_undergrad_names, 'courses': uh_undergrad_courses, 'descriptions': uh_undergrad_desc})

In [52]:
uh_df.to_csv('curriculum/uh_programs.csv', index=False)

## Saint Peter's

In [151]:
sp_base = "https://www.saintpeters.edu/academics/areas-of-study/?bachelors-degree&major&masters-degree"

In [152]:
sp_all = requests.get(sp_base).text
sp_soup = BeautifulSoup(sp_all, 'html.parser')

In [163]:
sp_undergrad_programs = [p for p in sp_soup.select('.program-links > a') + sp_soup.select('.program-links > div > a') if "undergrad" in p['href'] and "http" in p['href']]
sp_grad_programs = [p for p in sp_soup.select('.program-links > a') + sp_soup.select('.program-links > div > a') if "/grad" in p['href'] and "http" in p['href']]

### Undergrad

In [170]:
sp_undergrad_names = []
sp_undergrad_courses = []
sp_undergrad_desc = []
for p in tqdm(sp_undergrad_programs):
    if "about" in p.p.text:
        p_name = p.p.text[7:]
    else:
        p_name = p.p.text[4:]
    if requests.get(p['href'] + "/courses/").status_code == 200:
        p_courses = requests.get(p['href'] + "/courses/").text
        p_soup = BeautifulSoup(p_courses, 'html.parser')
        p_titles = ' '.join(list(map(lambda x: x.text.split(".  ")[1], p_soup.select('.courseblocktitle > strong'))))
        p_desc = ' '.join(list(map(lambda x: x.text.strip(), p_soup.select('.courseblockdesc'))))
        sp_undergrad_names.append(p_name)
        sp_undergrad_courses.append(p_titles)
        sp_undergrad_desc.append(p_desc)
    else:
        pass

100%|██████████| 68/68 [01:55<00:00,  1.70s/it]


In [214]:
sp_undergrad_df = pd.DataFrame({'institution': 'Saint Peter\'s', 'degree_level': 'undergraduate', 'program': sp_undergrad_names, 'courses': sp_undergrad_courses, 'descriptions': sp_undergrad_desc})
sp_undergrad_df = sp_undergrad_df[sp_undergrad_df['descriptions'] != ""].drop_duplicates(subset=['program'])

### Grad

In [201]:
sp_grad_names = []
sp_grad_courses = []
sp_grad_desc = []
for p in tqdm(sp_grad_programs):
    if "about" in p.p.text:
        p_name = p.p.text[7:]
    else:
        p_name = p.p.text[4:]
    if requests.get(p['href'] + "/courses/").status_code == 200:
        p_courses = requests.get(p['href'] + "/courses/").text
        p_soup = BeautifulSoup(p_courses, 'html.parser')
        p_titles = ' '.join(list(map(lambda x: x.text.split(".")[1].strip(), p_soup.select('.courseblocktitle > strong'))))
        p_desc = ' '.join(list(map(lambda x: x.text.strip(), p_soup.select('.courseblockdesc'))))
        sp_grad_names.append(p_name)
        sp_grad_courses.append(p_titles)
        sp_grad_desc.append(p_desc)
    else:
        pass

100%|██████████| 17/17 [00:29<00:00,  1.71s/it]


In [212]:
sp_grad_df = pd.DataFrame({'institution': 'Saint Peter\'s', 'degree_level': 'graduate', 'program': sp_grad_names, 'courses': sp_grad_courses, 'descriptions': sp_grad_desc})
sp_grad_df = sp_grad_df[sp_grad_df['descriptions'] != ""].drop_duplicates(subset=['program'])

In [219]:
sp_df = pd.concat([sp_undergrad_df, sp_grad_df]).reset_index(drop=True)

In [220]:
sp_df.to_csv('curriculum/sp_programs.csv', index=False)

# CIP Codes

## New Jersey

In [2]:
dir_url_nj = "https://www.state.nj.us/highereducation/Program_Inventory/PI_Search.shtml"

In [83]:
driver = webdriver.Chrome()
driver.get(dir_url_nj)

In [84]:
select = Select(driver.find_elements(By.NAME, 'words')[1])
select.select_by_visible_text('Princeton University')

In [85]:
driver.find_element(By.XPATH, "//input[@value='Search by Institution']").click()

In [86]:
pu_degree_ls = []
pu_program_ls = []
pu_cip_ls = []
n_pages = math.ceil(int(driver.find_elements(By.TAG_NAME, 'p')[2].text.split(' ')[0]) / 25)
for i in range(n_pages):
    rows = driver.find_elements(By.XPATH, '//center/table/tbody/tr/td/center[3]/table/tbody/tr')
    for r in rows:
        if "Degree" in r.text:
            if "bachelor" in r.text:
                pu_degree_ls.append("undergraduate")
            elif "master" in r.text or "doctoral" in r.text:
                pu_degree_ls.append("graduate")
            else:
                pu_degree_ls.append("other")
        elif "Program" in r.text:
            pu_program_ls.append(r.text.split(": ")[1])
        elif "CIP Code" in r.text:
            pu_cip_ls.append(re.search('(?<=: )(\d+)(?= -)', r.text)[0])
    if i < n_pages - 1:
        next = driver.find_element(By.XPATH, "//input[@value='Get Next Matches']")
        next.click()
    else:
        print("Reached last page")
driver.close()

Reached last page


In [87]:
pu_cip = pd.DataFrame({'institution': 'Princeton', 'degree_level': pu_degree_ls, 'program': pu_program_ls, 'cip': pu_cip_ls})

In [94]:
pu_curriculum = pd.read_csv('curriculum/princeton_programs.csv')

In [110]:
pu_merged = pu_curriculum.merge(pu_cip[pu_cip['degree_level'] == "undergraduate"].drop(columns=["institution"]), on="program", how="left")
pu_merged['degree_level'].fillna("undergraduate", inplace=True)

In [113]:
pu_merged.to_csv("cip/nj_full.csv", index=False)

## Indiana

In [114]:
dir_url_in = "https://transferin.net/transfer-resources/transfer-databases/academic-program-inventory/"

In [117]:
purdue_cip = pd.read_csv("cip/in_cip.csv", delimiter="\t")

In [122]:
purdue_cip.head(1)

,institution,cip,degree_level,program
0,Purdue,1.0,undergraduate,B.S. in Agriculture


In [149]:
purdue_cip['program'] = purdue_cip['program'].str.extract("((?<=in ).*)")

In [119]:
purdue_curriculum = pd.read_csv("curriculum/purdue_bach_programs.csv")

In [123]:
purdue_curriculum.head(1)

,institution,degree_level,program,courses,descriptions
0,purdue,undergraduate,"African American Studies, BA",Introduction To African American Studies The A...,Credit Hours: 3.00. Introduction to the philos...


In [147]:
purdue_curriculum['program'] = purdue_curriculum['program'].str.extract("(.*(?=,))")

In [151]:
purdue_merged = purdue_curriculum.merge(purdue_cip.drop(columns=["institution", "degree_level"]), on="program", how="left")

In [163]:
purdue_merged.to_csv("cip/in_full.csv", encoding='utf-8', index=False)

In [160]:
purdue_merged.head()

,institution,degree_level,program,courses,descriptions,cip
0,purdue,undergraduate,African American Studies,Introduction To African American Studies The A...,Credit Hours: 3.00. Introduction to the philos...,5.0201
1,purdue,undergraduate,American Studies,Interpreting America Perspectives On America S...,Credit Hours: 3.00. Introduction to the interd...,5.0102
2,purdue,undergraduate,Anthropology Honors,Introduction To Archaeology And World Prehisto...,Credit Hours: 3.00. Introduction to the ideas ...,NaN
3,purdue,undergraduate,Anthropology,Introduction To Archaeology And World Prehisto...,Credit Hours: 3.00. Introduction to the ideas ...,45.0201
4,purdue,undergraduate,Art History,History Of Art To 1400 History Of Art Since 14...,"Credit Hours: 3.00. Review of painting, sculpt...",50.0703
